In [1]:
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper
import dill
import numpy as np
from qiskit_algorithms.optimizers import SPSA, SLSQP, COBYLA
from qiskit_aer.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit.circuit.library import EfficientSU2

In [2]:
import qiskit_nature
qiskit_nature.__version__

'0.7.2'

In [3]:
BL = 0.6

In [4]:
# Parameters
BL = 2.65


### 1 Construct the Hamiltonian

Helper function to construct the Hamiltonian represented as a Pauli decomposition stored in `qubit_op`, while also returning the `nuclear_repulsion_energy` and `core_electron_energy`.

In [5]:
def get_qubit_op(BL):
    molecule = MoleculeInfo(
        symbols=["H", "H"],
        coords=([0.0, 0.0, 0.0], [BL, 0.0, 0.0]),
        multiplicity=1,
        charge=0,
    )

    driver = PySCFDriver.from_molecule(molecule)
    properties = driver.run()

    problem = FreezeCoreTransformer(
        freeze_core=True, remove_orbitals=[-3, -2]
    ).transform(properties)

    # Get the fermionic Hamiltonian (first element)
    fermionic_op, _ = problem.second_q_ops()

    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals

    # Use Jordan-Wigner (no tapering)
    mapper = JordanWignerMapper()
    qubit_op = mapper.map(fermionic_op)
    # converter = QubitConverter(mapper=mapper)
    # qubit_op = converter.convert(fermionic_op, num_particles=num_particles)

    core_electron_energy = problem.hamiltonian.constants["FreezeCoreTransformer"]
    nuclear_repulsion_energy = problem.nuclear_repulsion_energy

    return (
        qubit_op,
        num_particles,
        num_spatial_orbitals,
        problem,
        mapper,
        nuclear_repulsion_energy,
        core_electron_energy,
    )


In [6]:
(qubit_op, num_particles, num_spatial_orbitals, problem, mapper,nuclear_repulsion_energy, core_electron_energy) = get_qubit_op(BL)

In [7]:
qubit_op

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IIZZ', 'IZII', 'IZIZ', 'ZIII', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.74422544+0.j,  0.04982003+0.j,  0.02878913+0.j,  0.04981446+0.j,
  0.04982003+0.j,  0.12017998+0.j,  0.02878913+0.j,  0.12178485+0.j,
  0.07197039+0.j,  0.07197039+0.j,  0.07197039+0.j,  0.07197039+0.j,
  0.12178485+0.j,  0.12362853+0.j,  0.04981446+0.j])

In [8]:
# Extract terms and coefficients
pauli_list = list(zip(qubit_op.paulis, qubit_op.coeffs))

# Sort terms by absolute coefficient magnitude (largest first)
sorted_pauli_list = sorted(pauli_list, key=lambda x: abs(x[1]), reverse=True)

# Print sorted terms
# for pauli, coef in sorted_pauli_list:
#     print(f"Pauli: {pauli}, Coefficient: {coef}")

# Save the list
with open("sorted_pauli_list.pkl", "wb") as f:
    dill.dump(sorted_pauli_list, f)

In [9]:
dd = np.real(min(np.linalg.eigvals(qubit_op.to_matrix())))

### 2 Perform VQE

Define a class to perform VQE.

In [10]:
# Define the custom VQE
class CustomVQE:
    def __init__(self, estimator, ansatz, optimizer, qubit_op, dd, NREplusCEE, custom_function, initial_point=None):
        self.estimator = estimator
        self.ansatz = ansatz
        self.optimizer = optimizer
        self.qubit_op = qubit_op
        self.NREplusCEE = NREplusCEE
        self.dd = dd
        self.custom_function = custom_function
        self.initial_point = initial_point  or [0] * ansatz.num_parameters

    def compute_minimum_eigenvalue(self):
        # Define the function to minimize
        def objective_function(params):
            # Evaluate the expectation value of the qubit operator
            expectation = self.estimator.run([self.ansatz], [self.qubit_op], [params]).result().values[0]
            # Evaluate the custom objective function
            return self.custom_function(expectation, params)

        # Use the `minimize` method for optimization
        opt_result = self.optimizer.minimize(
            fun=objective_function,
            x0=self.initial_point
        )

        result = {
            'x': [float(value) for value in opt_result.x],
            'fun': opt_result.fun,
            'dd': self.dd,
            'NREplusCEE': self.NREplusCEE
        }

        
        return result


In [11]:
# l=12 and maxiter = 200 works for LiH and HF

In [12]:
optimizer = SLSQP(maxiter=300)
noiseless_estimator = Estimator(approximation=True)


# Define ansatz
var_form = EfficientSU2(qubit_op.num_qubits, reps=12, entanglement='full', skip_unentangled_qubits=False, parameter_prefix='a')


# Example Custom Objective Function
def custom_objective_function(expectation_value, parameters):
    # penalty_term = 0.1 * sum(param**2 for param in parameters)  # Example regularization
    return expectation_value 


# Create an instance of CustomVQE
custom_vqe = CustomVQE(
    estimator=noiseless_estimator,
    ansatz=var_form,
    optimizer=optimizer,
    qubit_op=qubit_op,
    dd = dd,
    NREplusCEE = nuclear_repulsion_energy + core_electron_energy,
    custom_function=custom_objective_function,
    initial_point = list(np.random.uniform(-np.pi, np.pi, var_form.num_parameters))
)


In [13]:
vqe_result = custom_vqe.compute_minimum_eigenvalue()

In [14]:
vqe_result

{'x': [0.3751214352688147,
  -2.357071985505295,
  1.9940000121582848,
  2.199839390105514,
  2.4445362430247384,
  2.3913847062673854,
  -1.4052823976639226,
  -2.7299423654675965,
  2.231245454097882,
  -3.096302282818919,
  -0.024706464297798273,
  0.11000497141883751,
  2.073904036459798,
  3.0071291311443376,
  -1.7148571031683444,
  -2.640436701598036,
  -1.5418752861616167,
  -1.2321245417681392,
  2.178733781452323,
  -1.505932539472427,
  -2.4264180477857207,
  1.2106985939299906,
  2.0162070666690424,
  -1.3751404480874185,
  -2.0535471548652566,
  1.2415015206924074,
  2.8057694870820304,
  3.096770071503636,
  -0.3953018562559686,
  -0.05481652645623327,
  2.1766367393121384,
  -1.2795272739878727,
  -1.764598585750181,
  1.0107145703468983,
  -1.2696385958586527,
  -2.154200752217113,
  0.42887354526749516,
  -2.629615125178459,
  0.8059219145699591,
  1.1298101492731263,
  2.4882476693634206,
  -1.5733938787865098,
  -2.093963254656009,
  2.0473607032900087,
  -2.94864022

In [15]:
len(vqe_result['x'])

104

In [16]:
print(vqe_result['fun'] - vqe_result['dd'])

3.974199075162943e-06


In [17]:
with open("vqe_result.pkl", "wb") as f:
    dill.dump(vqe_result, f)